# Sample Dataset

In [1]:
import pandas as pd

# Create a toy dataset with various data quality issues
data = {
    "Name": ["John Doe", "Jane Smith", "Johnathan Doe", "Jake Brown", "Jake Brown"],
    "Email": [
        "john@example.com",
        "jane@example,com",  
        "johnathan.example.com",  
        "jake.brown@example.com",
        "jake.brown@example.com"  
    ],
    "EyeColor": ["Blue", "Green", "Blue", "Pale", "Blue"],  
    "Age": [30, None, 50, 45, 45],  
    "Height": [173, 160, 15, 180, 180],  
    "DateOfBirth": [
        "1989-07-11",
        "1992-03-05",
        "1970-07-11",
        "1977-02-10",
        "1977-02-10" 
    ],
    "Address": [
        "123 Fake St",
        "456 Real Rd",
        "123 Fake St",
        "789 Another Pl",
        "789 Another Pl"
    ],
    "LastUpdated": [
        "2025-01-01",
        "2024-12-31",
        "2022-05-25",
        "2023-10-01",
        "2023-10-01"  
    ]
}

df = pd.DataFrame(data)
df


,Name,Email,EyeColor,Age,Height,DateOfBirth,Address,LastUpdated
0,John Doe,john@example.com,Blue,30.0,173,1989-07-11,123 Fake St,2025-01-01
1,Jane Smith,"jane@example,com",Green,NaN,160,1992-03-05,456 Real Rd,2024-12-31
2,Johnathan Doe,johnathan.example.com,Blue,50.0,15,1970-07-11,123 Fake St,2022-05-25
3,Jake Brown,jake.brown@example.com,Pale,45.0,180,1977-02-10,789 Another Pl,2023-10-01
4,Jake Brown,jake.brown@example.com,Blue,45.0,180,1977-02-10,789 Another Pl,2023-10-01


# Accuracy

**Demo: Checking Suspicious Height Values**

In our sample dataset, a 15 cm height for an adult is suspiciously low.

In [2]:
# Convert Height to numeric in case it's not
df["Height"] = pd.to_numeric(df["Height"], errors="coerce")

# Check for suspicious values: let's flag heights below 50 cm or above 250 cm
suspect_height = df[(df["Height"] < 50) | (df["Height"] > 250)]
suspect_height


,Name,Email,EyeColor,Age,Height,DateOfBirth,Address,LastUpdated
2,Johnathan Doe,johnathan.example.com,Blue,50.0,15,1970-07-11,123 Fake St,2022-05-25


# Completeness

**Demo: Checking Missing Values**

Let’s see which columns in our dataset have missing entries:

In [3]:
df.isnull().sum()


Name           0
Email          0
EyeColor       0
Age            1
Height         0
DateOfBirth    0
Address        0
LastUpdated    0
dtype: int64

# Uniqueness

**Demo: Finding Duplicate Rows**

We might consider a combination of columns—Name and DateOfBirth—as unique identifiers:

In [4]:
duplicates = df.duplicated(subset=["Name", "DateOfBirth"], keep=False)
df[duplicates]


,Name,Email,EyeColor,Age,Height,DateOfBirth,Address,LastUpdated
3,Jake Brown,jake.brown@example.com,Pale,45.0,180,1977-02-10,789 Another Pl,2023-10-01
4,Jake Brown,jake.brown@example.com,Blue,45.0,180,1977-02-10,789 Another Pl,2023-10-01


# Consistency

**Demo: Checking for Inconsistent Age and Date of Birth**

We can do a basic check to ensure that the reported Age and DateOfBirth align (with some tolerance).

In [5]:
import datetime

today = datetime.date(2025, 3, 25)

def calculate_age(dob_str):
    dob = datetime.datetime.strptime(dob_str, "%Y-%m-%d").date()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

df["CalculatedAge"] = df["DateOfBirth"].apply(calculate_age)
df["AgeDifference"] = df["CalculatedAge"] - df["Age"].fillna(-9999)

# Check if difference is more than 1 year
inconsistent_age = df[df["AgeDifference"].abs() > 1]
inconsistent_age


,Name,Email,EyeColor,Age,Height,DateOfBirth,Address,LastUpdated,CalculatedAge,AgeDifference
0,John Doe,john@example.com,Blue,30.0,173,1989-07-11,123 Fake St,2025-01-01,35,5.0
1,Jane Smith,"jane@example,com",Green,NaN,160,1992-03-05,456 Real Rd,2024-12-31,33,10032.0
2,Johnathan Doe,johnathan.example.com,Blue,50.0,15,1970-07-11,123 Fake St,2022-05-25,54,4.0
3,Jake Brown,jake.brown@example.com,Pale,45.0,180,1977-02-10,789 Another Pl,2023-10-01,48,3.0
4,Jake Brown,jake.brown@example.com,Blue,45.0,180,1977-02-10,789 Another Pl,2023-10-01,48,3.0


# Timeliness

**Demo: Checking Data Freshness**

Let’s say we want our records to have been updated within the last year (relative to 2025-03-25).

In [6]:
df["LastUpdated"] = pd.to_datetime(df["LastUpdated"], errors="coerce")

threshold_date = pd.to_datetime("2024-03-26")
df["Timely"] = df["LastUpdated"] >= threshold_date
df

,Name,Email,EyeColor,Age,Height,DateOfBirth,Address,LastUpdated,CalculatedAge,AgeDifference,Timely
0,John Doe,john@example.com,Blue,30.0,173,1989-07-11,123 Fake St,2025-01-01,35,5.0,True
1,Jane Smith,"jane@example,com",Green,NaN,160,1992-03-05,456 Real Rd,2024-12-31,33,10032.0,True
2,Johnathan Doe,johnathan.example.com,Blue,50.0,15,1970-07-11,123 Fake St,2022-05-25,54,4.0,False
3,Jake Brown,jake.brown@example.com,Pale,45.0,180,1977-02-10,789 Another Pl,2023-10-01,48,3.0,False
4,Jake Brown,jake.brown@example.com,Blue,45.0,180,1977-02-10,789 Another Pl,2023-10-01,48,3.0,False


# Validity

**Demo: Checking Email and Eye Color Formats**

We’ll do a simple validation for the presence of ‘@’ in emails and look for eye colors in a controlled set.

In [7]:
# Basic email check
df["ValidEmail"] = df["Email"].apply(lambda x: "@" in x)

# Eye color validity
valid_eye_colors = ["Blue", "Green", "Brown", "Hazel"]
df["ValidEyeColor"] = df["EyeColor"].isin(valid_eye_colors)

df[["Name", "Email", "EyeColor", "ValidEmail", "ValidEyeColor"]]


,Name,Email,EyeColor,ValidEmail,ValidEyeColor
0,John Doe,john@example.com,Blue,True,True
1,Jane Smith,"jane@example,com",Green,True,True
2,Johnathan Doe,johnathan.example.com,Blue,False,True
3,Jake Brown,jake.brown@example.com,Pale,True,False
4,Jake Brown,jake.brown@example.com,Blue,True,True


# Building a Data Quality Class

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any, Optional

class DataQualityChecker:
    """
    A generic class to perform basic data quality checks on a Pandas DataFrame.
    Extend or customize methods to suit your domain-specific needs.
    """
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.results = {}
    
    def check_completeness(self) -> Dict[str, Any]:
        """
        Checks for missing values in each column.
        Returns a dictionary with the count and percentage of missing values.
        """
        missing_counts = self.df.isnull().sum()
        total_rows = len(self.df)
        missing_percentages = (missing_counts / total_rows) * 100
        
        return {
            "missing_counts": missing_counts.to_dict(),
            "missing_percentage": missing_percentages.round(2).to_dict()
        }

    def check_uniqueness(self, subset: Optional[list] = None) -> Dict[str, Any]:
        """
        Checks for duplicates in the DataFrame.
        By default, it considers the entire row. 
        You can provide a list of columns in `subset` to define duplicates differently.
        """
        if subset is None:
            subset = self.df.columns.tolist()
        
        total_rows = len(self.df)
        duplicate_flags = self.df.duplicated(subset=subset, keep=False)
        duplicates_count = duplicate_flags.sum()
        
        return {
            "duplicate_count": int(duplicates_count),
            "duplicate_percentage": round((duplicates_count / total_rows) * 100, 2),
            "subset_used": subset
        }

    def check_validity(self, validity_rules: Optional[dict] = None) -> Dict[str, Any]:
        """
        Checks whether columns conform to expected formats or ranges.
        
        Example `validity_rules`:
        {
            "Email": {"regex": r"[^@]+@[^@]+\.[^@]+"},
            "Height": {"range": (50, 250)}
        }
        
        You can extend this with more sophisticated checks or multiple rules per column.
        """
        if validity_rules is None:
            validity_rules = {}
        
        invalid_counts = {}
        for col, rules in validity_rules.items():
            invalid_indices = set()
            
            # Check against a regex (e.g., email format)
            if "regex" in rules:
                pattern = rules["regex"]
                # Convert to string to avoid errors on non-string or NaN
                regex_mask = self.df[col].astype(str).str.match(pattern, na=False)
                invalid_indices.update(self.df[~regex_mask].index)
            
            # Check numeric ranges
            if "range" in rules and pd.api.types.is_numeric_dtype(self.df[col]):
                min_val, max_val = rules["range"]
                range_mask = self.df[col].between(min_val, max_val, inclusive='both')
                invalid_indices.update(self.df[~range_mask].index)
            
            invalid_counts[col] = len(invalid_indices)
        
        return {"invalid_counts": invalid_counts}

    def check_accuracy(self, accuracy_rules: Optional[dict] = None) -> Dict[str, Any]:
        """
        A placeholder for domain-specific Accuracy checks.
        
        Example `accuracy_rules`:
        {
            "Height": {"range": (50, 250)}
        }
        
        Similar to validity but intended to ensure "real-world" plausibility.
        """
        if accuracy_rules is None:
            accuracy_rules = {}
        
        inaccurate_counts = {}
        for col, rules in accuracy_rules.items():
            invalid_indices = set()
            
            # Range-based "plausibility" check
            if "range" in rules and pd.api.types.is_numeric_dtype(self.df[col]):
                min_val, max_val = rules["range"]
                in_range_mask = self.df[col].between(min_val, max_val, inclusive='both')
                invalid_indices.update(self.df[~in_range_mask].index)
            
            # Non-negative check (example)
            if "non_negative" in rules and rules["non_negative"]:
                neg_mask = self.df[col] < 0
                invalid_indices.update(self.df[neg_mask].index)
            
            inaccurate_counts[col] = len(invalid_indices)
        
        return {"inaccurate_counts": inaccurate_counts}

    def check_consistency_age_dob(
        self, 
        age_col: str = "Age", 
        dob_col: str = "DateOfBirth", 
        reference_date: Optional[datetime] = None,
        allowed_tolerance_years: int = 1
    ) -> Dict[str, Any]:
        """
        Checks for consistency between an Age column and a DateOfBirth column.
        Flags rows where |calculated_age - recorded_age| > allowed_tolerance_years.
        """
        if reference_date is None:
            # Default reference date for the sake of example
            reference_date = datetime(2025, 3, 25)
        
        # Ensure DateOfBirth is in datetime
        self.df[dob_col] = pd.to_datetime(self.df[dob_col], errors='coerce')
        
        def calc_age(dob):
            if pd.isnull(dob):
                return np.nan
            years = reference_date.year - dob.year
            # subtract 1 year if the month/day hasn't been reached yet this year
            if (reference_date.month, reference_date.day) < (dob.month, dob.day):
                years -= 1
            return years
        
        calculated_ages = self.df[dob_col].apply(calc_age)
        
        # Compare with recorded Age (treat missing Age as -9999 to ensure difference is large)
        recorded_ages = self.df[age_col].fillna(-9999)
        diffs = (calculated_ages - recorded_ages).abs()
        
        inconsistent_mask = diffs > allowed_tolerance_years
        inconsistent_count = inconsistent_mask.sum()
        
        return {
            "inconsistent_count": int(inconsistent_count),
            "inconsistent_indices": self.df[inconsistent_mask].index.tolist()
        }

    def check_timeliness(
        self, 
        date_col: str = "LastUpdated", 
        threshold_date: Optional[datetime] = None
    ) -> Dict[str, Any]:
        """
        Checks if data in `date_col` is up-to-date relative to a threshold date.
        """
        if threshold_date is None:
            # Default threshold date for the sake of example
            threshold_date = datetime(2024, 1, 1)
        
        self.df[date_col] = pd.to_datetime(self.df[date_col], errors='coerce')
        
        outdated_mask = self.df[date_col] < threshold_date
        outdated_count = outdated_mask.sum()
        
        return {
            "threshold_date": threshold_date.strftime("%Y-%m-%d"),
            "outdated_count": int(outdated_count),
            "outdated_percentage": round(outdated_count / len(self.df) * 100, 2)
        }
    
    def run_all_checks(
        self,
        completeness: bool = True,
        uniqueness: bool = True,
        validity_rules: Optional[dict] = None,
        accuracy_rules: Optional[dict] = None,
        do_consistency_age_dob: bool = True,
        timeliness_args: Optional[dict] = None
    ) -> Dict[str, Any]:
        """
        Runs the selected checks and compiles results into a dictionary.
        You can tweak which checks to run with the boolean flags and arguments.
        
        :param do_consistency_age_dob: If True, runs the Age vs DOB consistency check
        :param timeliness_args: Dictionary specifying 'date_col' and 'threshold_date' (optional)
        """
        if completeness:
            self.results["completeness"] = self.check_completeness()
        if uniqueness:
            self.results["uniqueness"] = self.check_uniqueness()
        if validity_rules is not None:
            self.results["validity"] = self.check_validity(validity_rules=validity_rules)
        if accuracy_rules is not None:
            self.results["accuracy"] = self.check_accuracy(accuracy_rules=accuracy_rules)
        if do_consistency_age_dob:
            self.results["consistency_age_vs_dob"] = self.check_consistency_age_dob()
        if timeliness_args is not None:
            self.results["timeliness"] = self.check_timeliness(
                date_col=timeliness_args.get("date_col", "LastUpdated"),
                threshold_date=timeliness_args.get("threshold_date", datetime(2024, 1, 1))
            )
        
        return self.results





In [9]:
# EXAMPLE USAGE
if __name__ == "__main__":
    data = {
        "Name": ["John Doe", "Jane Smith", "Johnathan Doe", "Jake Brown", "Jake Brown"],
        "Email": [
            "john@example.com",
            "jane@example,com",  # invalid format
            "johnathan.example.com",  # missing '@'
            "jake.brown@example.com",
            "jake.brown@example.com"  # duplicate
        ],
        "Age": [30, None, 50, 45, 45],  # Missing for Jane Smith
        "Height": [173, 160, 15, 180, 180],  # 15 cm is suspicious
        "DateOfBirth": [
            "1989-07-11",
            "1992-03-05",
            "1970-07-11",
            "1977-02-10",
            "1977-02-10"
        ],
        "LastUpdated": [
            "2025-01-01",
            "2024-12-31",
            "2022-05-25",
            "2023-10-01",
            "2023-10-01"
        ]
    }

    df_sample = pd.DataFrame(data)
    dq_checker = DataQualityChecker(df_sample)

    # Define basic rules
    validity_rules = {
        "Email": {"regex": r"[^@]+@[^@]+\.[^@]+"},  # simplistic email check
        "Height": {"range": (50, 250)},            # plausible human height
    }
    accuracy_rules = {
        "Height": {"range": (50, 250)}             # real-world plausible height range
    }
    timeliness_args = {
        "date_col": "LastUpdated",
        "threshold_date": datetime(2024, 3, 25)
    }

    results = dq_checker.run_all_checks(
        completeness=True,
        uniqueness=True,
        validity_rules=validity_rules,
        accuracy_rules=accuracy_rules,
        do_consistency_age_dob=True,
        timeliness_args=timeliness_args
    )

    for dimension, outcome in results.items():
        print(f"--- {dimension.upper()} ---")
        print(outcome)
        print()

--- COMPLETENESS ---
{'missing_counts': {'Name': 0, 'Email': 0, 'Age': 1, 'Height': 0, 'DateOfBirth': 0, 'LastUpdated': 0}, 'missing_percentage': {'Name': 0.0, 'Email': 0.0, 'Age': 20.0, 'Height': 0.0, 'DateOfBirth': 0.0, 'LastUpdated': 0.0}}

--- UNIQUENESS ---
{'duplicate_count': 2, 'duplicate_percentage': 40.0, 'subset_used': ['Name', 'Email', 'Age', 'Height', 'DateOfBirth', 'LastUpdated']}

--- VALIDITY ---
{'invalid_counts': {'Email': 2, 'Height': 1}}

--- ACCURACY ---
{'inaccurate_counts': {'Height': 1}}

--- CONSISTENCY_AGE_VS_DOB ---
{'inconsistent_count': 5, 'inconsistent_indices': [0, 1, 2, 3, 4]}

--- TIMELINESS ---
{'threshold_date': '2024-03-25', 'outdated_count': 3, 'outdated_percentage': 60.0}

